## Install the dependecies

- ivy `ivy library`
- ivy_models `ivy models library`
- transformers ` Transformers library yo get the pretrained model`

**If you have all of this libraries installed you can save some time and skip this cell if not you should run this cell**

In [1]:
!pip install -q ivy

!git clone https://github.com/unifyai/models.git
%cd models
!python3 -m pip install -q -e .

!python3 -m pip install transformers

exit()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.6/45.6 kB 3.1 MB/s eta 0:00:00
Cloning into 'models'...
remote: Enumerating objects: 1701, done.
remote: Counting objects: 100% (530/530), done.
remote: Compressing objects: 100% (177/177), done.
remote: Total 1701 (delta 423), reused 384 (delta 346), pack-reused 1171
Receiving objects: 100% (1701/1701), 287.17 MiB | 19.97 MiB/s, done.
Resolving deltas: 100% (858/858), done.
/content/models
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 26.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 37.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 43.5 MB/s eta 0:00:00


## Import the modules

In [1]:
import torch
import ivy
import ivy_models
from transformers import AutoModel, AutoTokenizer

## Models inference

### First load the pretrained Model and tokenizer

In [2]:
bert_base = AutoModel.from_pretrained("bert-base-uncased")
bert_base = bert_base.eval() # for inference and evaluation
bert_tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

### Then load ivy model

In [3]:
# Don't worry about the warnings it's mainly related to ivy backend
# You could avoid them by using ivy.set_backend(your_backend)
ivy_bert = ivy_models.bert_base_uncased(pretrained=True)

### Prepare the inputs to bert model

In [4]:
inputs_text = "what a beautiful day!"
inputs = bert_tokenizer(inputs_text, return_tensors='pt')

### Transformers output

In [5]:
with torch.no_grad():
   bert_output = bert_base(**inputs).pooler_output

### Compile ivy Model before inference

In [7]:
ivy.set_backend('numpy')
ivy_inputs = {k:ivy.asarray(v.numpy()) for k, v in inputs.items()}
ivy_bert.compile(kwargs=ivy_inputs)

### Ivy inference

In [12]:
ivy_output = ivy_bert(**ivy_inputs)['pooler_output']

## Sequnce Classification
**We compare the logits of the pooler output from each model** (shape and values)

In [13]:
import numpy as np
print(f"logits shapes, Ivy: {list(ivy_output.shape)}, torch: {list(bert_output.shape)}")

logits shapes, Ivy: [1, 768], torch:[1, 768]


In [14]:
np.allclose(ivy_output, bert_output.detach().numpy(),rtol=0.005,atol=0.005)

True